In [1]:
# Import required packages
import os
import shutil

import keras.callbacks as kc
import keras.layers as kl
import keras.models as km
import keras.optimizers as ko
import keras.utils as ku
import numpy as np
import sklearn as sk
import sklearn.utils as sku

import Config as conf
import CSV as csv

Using TensorFlow backend.


In [2]:
# Check output directory
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    conf.N_HIDDEN, "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              conf.N_HIDDEN, "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)

In [3]:
# Setup Keras Callbacks
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)

If you have tensorboard in this environment, you can type below to see the result in tensorboard:
    tensorboard --logdir=./Log_LR1e-05_BATCH16_HIDDEN1024/


In [4]:
# Notice Checkpoint Directory
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

Keras checkpoints and final result will be saved in here:
    ./Output_LR1e-05_BATCH16_HIDDEN1024/


In [5]:
# Setup Keras RNN Model
lstm = kl.LSTM(
    conf.N_HIDDEN,
    input_shape=(conf.WINDOW_SIZE, conf.N_COLUMNS))
lstm.add_loss(1e-8)
adam = ko.Adam(lr=conf.LEARNING_RATE, amsgrad=True)
model = km.Sequential()
model.add(lstm)
model.add(kl.Dense(
    conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES, activation="softmax"))
model.compile(
    loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Import CSV data
xs, ys = csv.getCSV()

Input directory not found. Calculate merged CSVs...
About noa ...
Processing X:  ./Dataset\csi_noa_1.csv
Processing X:  ./Dataset\csi_noa_2.csv
Processing Y:  ./Dataset\action_noa_1.csv
Processing Y:  ./Dataset\action_noa_2.csv
(1092, 169200) (1092, 3)
Writing calculated X/Ys ...
==== Action "noa" Finished! ====
About syncope ...
Processing X:  ./Dataset\csi_syncope_1.csv
Processing X:  ./Dataset\csi_syncope_2.csv
Processing X:  ./Dataset\csi_syncope_3.csv
Processing X:  ./Dataset\csi_syncope_4.csv
Processing X:  ./Dataset\csi_syncope_5.csv
Processing X:  ./Dataset\csi_syncope_6.csv
Processing X:  ./Dataset\csi_syncope_7.csv
Processing X:  ./Dataset\csi_syncope_8.csv
Processing X:  ./Dataset\csi_syncope_9.csv
Processing Y:  ./Dataset\action_syncope_1.csv
Processing Y:  ./Dataset\action_syncope_2.csv
Processing Y:  ./Dataset\action_syncope_3.csv
Processing Y:  ./Dataset\action_syncope_4.csv
Processing Y:  ./Dataset\action_syncope_5.csv
Processing Y:  ./Dataset\action_syncope_6.csv
Proce

In [7]:
# Shuffle data
for a in conf.ACTIONS:
    xs[a], ys[a] = sku.shuffle(xs[a], ys[a], random_state=0)

In [ ]:
# Run KFold
for i in range(conf.KFOLD):
    # Roll the data
    for a in conf.ACTIONS:
        xs[a] = np.roll(xs[a], int(len(xs[a]) / conf.KFOLD), axis=0)
        ys[a] = np.roll(ys[a], int(len(ys[a]) / conf.KFOLD), axis=0)

    # Data separation
    xTrain = []
    yTrain = []
    xEval = []
    yEval = []
    for a in conf.ACTIONS:
        if xTrain == []:
            xTrain = xs[a][int(len(xs[a]) / conf.KFOLD):]
        else:
            xTrain = np.r_[xTrain, xs[a][int(len(xs[a]) / conf.KFOLD):]]
        if yTrain == []:
            yTrain = ys[a][int(len(ys[a]) / conf.KFOLD):]
        else:
            yTrain = np.r_[yTrain, ys[a][int(len(ys[a]) / conf.KFOLD):]]
        if xEval == []:
            xEval = xs[a][:int(len(xs[a]) / conf.KFOLD)]
        else:
            xEval = np.r_[xEval, xs[a][:int(len(xs[a]) / conf.KFOLD)]]
        if yEval == []:
            yEval = ys[a][:int(len(ys[a]) / conf.KFOLD)]
        else:
            yEval = np.r_[yEval, ys[a][:int(len(ys[a]) / conf.KFOLD)]]

    if not conf.USE_NOACTIVITY:
        # Remove NoActivity from ys
        yTrain = yTrain[:, 1:]
        yEval = yEval[:, 1:]

        # If there exists only one action, convert Y to binary form
        if yEval.shape[1] == 1:
            yTrain = ku.to_categorical(yTrain)
            yEval = ku.to_categorical(yEval)

    # Setup Keras Checkpoint
    checkpoint = kc.ModelCheckpoint(
        outputDir + "Checkpoint_K" + str(i + 1) + "_EPOCH{epoch}_ACC{val_acc:.6f}.h5", period=25)

    # Fit model (learn)
    print(
        str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) +
        " th.")

    model.fit(
        xTrain,
        yTrain,
        batch_size=conf.BATCH_SIZE,
        epochs=conf.N_ITERATIONS,
        verbose=1,
        callbacks=[tensorboard, checkpoint],
        validation_data=(xEval, yEval))  #, validation_freq=2)
print("Epoch completed!")

C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


1 th fitting started. Endpoint is 11 th.
Instructions for updating:
Use tf.cast instead.


C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\chish\.conda\envs\ppp\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Train on 971 samples, validate on 96 samples
Epoch 1/128
971/971 [==============================] - 179s 185ms/step - loss: 0.3727 - acc: 0.8239 - val_loss: 0.1729 - val_acc: 0.9375
Epoch 2/128
971/971 [==============================] - 177s 182ms/step - loss: 0.1642 - acc: 0.9382 - val_loss: 0.1418 - val_acc: 0.9375
Epoch 3/128
971/971 [==============================] - 175s 180ms/step - loss: 0.1375 - acc: 0.9464 - val_loss: 0.1246 - val_acc: 0.9375
Epoch 4/128
971/971 [==============================] - 175s 180ms/step - loss: 0.1193 - acc: 0.9547 - val_loss: 0.1165 - val_acc: 0.9479
Epoch 5/128
971/971 [==============================] - 174s 179ms/step - loss: 0.1094 - acc: 0.9588 - val_loss: 0.1084 - val_acc: 0.9479
Epoch 6/128
971/971 [==============================] - 174s 180ms/step - loss: 0.0983 - acc: 0.9609 - val_loss: 0.1015 - val_acc: 0.9583
Epoch 7/128
971/971 [==============================] - 175s 180ms/step - loss: 0.0892 - acc: 0.9701 - val_loss: 0.0914 - val_acc: 0.9

Epoch 60/128
971/971 [==============================] - 175s 180ms/step - loss: 7.2471e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 61/128
971/971 [==============================] - 175s 180ms/step - loss: 7.0340e-04 - acc: 1.0000 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 62/128
971/971 [==============================] - 175s 180ms/step - loss: 6.8294e-04 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 1.0000
Epoch 63/128
971/971 [==============================] - 175s 180ms/step - loss: 6.6294e-04 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 1.0000
Epoch 64/128
971/971 [==============================] - 175s 180ms/step - loss: 6.4418e-04 - acc: 1.0000 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 65/128
971/971 [==============================] - 174s 180ms/step - loss: 6.2450e-04 - acc: 1.0000 - val_loss: 0.0046 - val_acc: 1.0000
Epoch 66/128
971/971 [==============================] - 174s 179ms/step - loss: 6.0346e-04 - acc: 1.0000 - val_loss: 0.0045 - val_acc: 1.0000
Epoch 

In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

In [ ]:
# Finished